In [1]:
import cdsapi
import os
import zipfile
import pandas as pd

In [2]:
# Definieren Sie den Namen des Unterordners für den Download und das Entpacken
output_folder = "CDS_data"
# Erstellen Sie den Ordner, falls er noch nicht existiert
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Dateipfad für die heruntergeladene ZIP-Datei
zip_file_path = os.path.join(output_folder, "daten.zip")

# API-Abfrage
dataset = "sis-energy-derived-reanalysis"
request = {
    "variable": [
        "wind_speed_at_100m",
        "wind_speed_at_10m",
        "surface_downwelling_shortwave_radiation",
        "pressure_at_sea_level",
        "2m_air_temperature",
        "total_precipitation"
    ],
    "spatial_aggregation": ["country_level"],
    "temporal_aggregation": ["hourly"]
}

client = cdsapi.Client()
client.retrieve(dataset, request).download(zip_file_path)

# ZIP-Datei entpacken und CSV-Dateien in den gleichen Ordner speichern
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_folder)

# Optional: ZIP-Datei nach dem Entpacken löschen
os.remove(zip_file_path)

print(f"Die Dateien wurden entpackt und befinden sich im Ordner: {output_folder}")

2025-03-13 16:22:25,435 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-03-13 16:22:25,436 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-03-13 16:22:25,927 WARNING [2025-02-10T00:00:00] This dataset is soon to be deprecated. An improved climate and energy indicators derived from reanalysis dataset at global level is currently in preparation for publication in the next few months. Watch our [Forum](https://forum.ecmwf.int/) for a future announcement!
2025-03-13 16:22:25,927 INFO Request ID is 8843385f-f539-4214-8e83-2e914b98e44d
2025-03-13 16:22:26,025 INFO status has been updated to accepted
2025-03-13 16:22:31,053 INFO status has been updated to running
2025-03-13 16:22:34,52

7985c98c747829472b6d7a99a1a0aa76.zip:   0%|          | 0.00/344M [00:00<?, ?B/s]

Die Dateien wurden entpackt und befinden sich im Ordner: CDS_data


In [3]:
# Mapping zwischen Indikator und DataFrame-Name
indicator_to_df_name = {
    'GHI_0000m': 'df_ghi',
    'TA-_0002m': 'df_temperature',
    'TP-_0000m': 'df_rain',
    'WS-_0010m': 'df_wind_speed_10m',
    'WS-_0100m': 'df_wind_speed_100m'
}

# Pfad zum Unterordner 'source'
source_folder = 'CDS_data'

# Liste der Dateien im Ordner 'source'
files = [f for f in os.listdir(source_folder) if f.endswith('.csv')]

# DataFrames in einem Dictionary speichern
dataframes = {}

# Dateien einlesen und DataFrames erstellen
for file in files:
    for indicator, df_name in indicator_to_df_name.items():
        if indicator in file:
            # CSV-Datei einlesen und erste 52 Zeilen überspringen
            df = pd.read_csv(os.path.join(source_folder, file), skiprows=52)
            dataframes[df_name] = df
            print(f"DataFrame '{df_name}' wurde erstellt.")
            break

# Beispiel, wie man auf einen bestimmten DataFrame zugreift
# print(dataframes['df_electricity_demand'].head())

# DataFrames als Variablen definieren
for df_name, df in dataframes.items():
    globals()[df_name] = df


DataFrame 'df_ghi' wurde erstellt.
DataFrame 'df_ghi' wurde erstellt.
DataFrame 'df_temperature' wurde erstellt.
DataFrame 'df_temperature' wurde erstellt.
DataFrame 'df_rain' wurde erstellt.
DataFrame 'df_rain' wurde erstellt.
DataFrame 'df_wind_speed_10m' wurde erstellt.
DataFrame 'df_wind_speed_10m' wurde erstellt.
DataFrame 'df_wind_speed_100m' wurde erstellt.
DataFrame 'df_wind_speed_100m' wurde erstellt.


In [4]:
# Liste der DataFrames und Namen ohne Präfix "df_"
dataframes = {
    'ghi': df_ghi,
    'temperature': df_temperature,
    'rain': df_rain,
    'wind_speed_10m': df_wind_speed_10m,
    'wind_speed_100m': df_wind_speed_100m}

# Konvertieren und Setzen des Indexes für jeden DataFrame
for name, df in dataframes.items():
    df.rename(columns={'Date': 'Datetime'}, inplace=True)  # Umbenennen der Datumsspalte
    df['Datetime'] = pd.to_datetime(df['Datetime'])        # Konvertieren in datetime
    
    df.set_index('Datetime', inplace=True)                 # Setzen als Index


In [5]:
# Erstellen des finalen DataFrames
df_deutschland = pd.DataFrame()

for name, df in dataframes.items():
    df_deutschland[name] = df['DE']


df_deutschland = df_deutschland.tz_localize('UTC')
df_deutschland = df_deutschland.tz_convert('Europe/Berlin')
    
df_deutschland["temperature"] = df_deutschland["temperature"]-273.15
df_deutschland = df_deutschland.loc["2012-01-01":]

In [6]:
df_deutschland.describe()

,ghi,temperature,rain,wind_speed_10m,wind_speed_100m
count,119425.000000,119423.000000,1.194250e+05,119423.000000,119423.000000
mean,127.062273,9.742829,9.869656e-05,3.251522,5.585726
std,189.425574,7.612359,1.454968e-04,1.320276,2.151227
min,-0.000528,-16.866430,-1.734723e-18,0.747314,0.971502
25%,0.000000,3.832670,5.113034e-06,2.271016,4.030522
50%,5.725749,9.218400,3.846480e-05,2.963270,5.225388
75%,205.171810,15.364510,1.322301e-04,3.937875,6.757012
max,879.099700,34.843160,2.068481e-03,10.966816,17.036156


In [7]:
df_deutschland.to_csv("germany_weather.csv", index = True)

In [8]:
df_deutschland.loc['2024-10-26':'2024-10-27']
df_deutschland.index.T

DatetimeIndex(['2012-01-01 00:00:00+01:00', '2012-01-01 01:00:00+01:00',
               '2012-01-01 02:00:00+01:00', '2012-01-01 03:00:00+01:00',
               '2012-01-01 04:00:00+01:00', '2012-01-01 05:00:00+01:00',
               '2012-01-01 06:00:00+01:00', '2012-01-01 07:00:00+01:00',
               '2012-01-01 08:00:00+01:00', '2012-01-01 09:00:00+01:00',
               ...
               '2025-02-28 15:00:00+01:00', '2025-02-28 16:00:00+01:00',
               '2025-02-28 17:00:00+01:00', '2025-02-28 18:00:00+01:00',
               '2025-02-28 19:00:00+01:00', '2025-02-28 20:00:00+01:00',
               '2025-02-28 21:00:00+01:00', '2025-02-28 22:00:00+01:00',
               '2025-02-28 23:00:00+01:00', '2025-03-01 00:00:00+01:00'],
              dtype='datetime64[ns, Europe/Berlin]', name='Datetime', length=119425, freq=None)

In [9]:
df_deutschland.loc['2024-11-01':'2024-11-02']

,ghi,temperature,rain,wind_speed_10m,wind_speed_100m
Datetime,,,,,
2024-11-01 00:00:00+01:00,6.467518e-14,9.42310,0.000004,2.928888,5.281851
2024-11-01 01:00:00+01:00,-6.467518e-14,9.26528,0.000005,3.052070,5.486490
2024-11-01 02:00:00+01:00,-6.467518e-14,9.12930,0.000005,3.129703,5.556750
2024-11-01 03:00:00+01:00,-6.467518e-14,9.00677,0.000006,3.146558,5.559964
2024-11-01 04:00:00+01:00,-6.467518e-14,8.90762,0.000006,3.188831,5.591042
2024-11-01 05:00:00+01:00,-6.467518e-14,8.89600,0.000005,3.152104,5.511374
2024-11-01 06:00:00+01:00,-6.467518e-14,8.86870,0.000007,3.191458,5.618847
2024-11-01 07:00:00+01:00,7.248466e-05,8.85010,0.000007,3.256147,5.672367
2024-11-01 08:00:00+01:00,1.348693e+01,8.97910,0.000012,3.426197,5.778727
